In [ ]:
# Import necessary libraries
import json
from typing import Dict, List, Optional
from fastapi import FastAPI, HTTPException, Query, Depends
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator
from pprint import pprint
import uvicorn

In [ ]:
# Define the Patient class
class Patient:
    def __init__(self, name, age, height, weight, lab_results=None, doctors_notes="", severity="low", patient_id=None):
        self.name = name
        self.age = age
        self.height = height
        self.weight = weight
        self.lab_results = lab_results or {}
        self.doctors_notes = doctors_notes
        self.severity = severity
        self.patient_id = patient_id or self._generate_id()

    def _generate_id(self):
        # Simple ID generation for demonstration
        import uuid
        return str(uuid.uuid4())[:8]

    def to_dict(self):
        return {
            "patient_id": self.patient_id,
            "name": self.name,
            "age": self.age,
            "height": self.height,
            "weight": self.weight,
            "lab_results": self.lab_results,
            "doctors_notes": self.doctors_notes,
            "severity": self.severity
        }

In [ ]:
# Create the PatientManager class for handling patient data
class PatientManager:
    def __init__(self, verbose=False):
        self.patients = {}
        self.verbose = verbose
        if verbose:
            print("Patient Manager initialized")

    def add_patient(self, patient):
        if patient.patient_id in self.patients:
            return {"success": False, "error": "Patient ID already exists"}

        self.patients[patient.patient_id] = patient

        if self.verbose:
            print(f"Added patient: {patient.name} with ID: {patient.patient_id}")

        return {"success": True, "patient_id": patient.patient_id}

    def get_patient(self, patient_id):
        return self.patients.get(patient_id)

    def get_all_patients(self, skip=0, limit=100):
        patients_list = list(self.patients.values())
        return patients_list[skip:skip+limit]

    def count_patients(self):
        return len(self.patients)

    def update_patient(self, patient_id, update_data):
        patient = self.get_patient(patient_id)
        if not patient:
            return {"success": False, "error": "Patient not found"}

        # Validate data
        errors = {}
        if "age" in update_data and (update_data["age"] < 0 or update_data["age"] > 120):
            errors["age"] = "Age must be between 0 and 120"

        if "height" in update_data and (update_data["height"] < 0 or update_data["height"] > 300):
            errors["height"] = "Height must be between 0 and 300 cm"

        if "weight" in update_data and (update_data["weight"] < 0 or update_data["weight"] > 500):
            errors["weight"] = "Weight must be between 0 and 500 kg"

        if "severity" in update_data and update_data["severity"] not in ["low", "medium", "high"]:
            errors["severity"] = "Severity must be one of: low, medium, high"

        if errors:
            return {"success": False, "errors": errors}

        # Update patient attributes
        for key, value in update_data.items():
            if hasattr(patient, key):
                setattr(patient, key, value)

        if self.verbose:
            print(f"Updated patient: {patient.name}")

        return {"success": True, "patient_id": patient_id}

    def delete_patient(self, patient_id):
        if patient_id not in self.patients:
            return {"success": False, "error": "Patient not found"}

        del self.patients[patient_id]

        if self.verbose:
            print(f"Deleted patient with ID: {patient_id}")

        return {"success": True}

    def search_patients(self, query):
        query = query.lower()
        results = []

        for patient in self.patients.values():
            if query in patient.name.lower():
                results.append(patient)

        return results

    def filter_patients(self, criteria):
        results = []

        for patient in self.patients.values():
            match = True

            if "min_age" in criteria and patient.age < criteria["min_age"]:
                match = False

            if "max_age" in criteria and patient.age > criteria["max_age"]:
                match = False

            if "severity" in criteria and patient.severity != criteria["severity"]:
                match = False

            if match:
                results.append(patient)

        return results

In [ ]:
# Define async functions to simulate asynchronous database operations
async def async_create_patient(name, age, height, weight, lab_results=None, doctors_notes="", severity="low"):
    patient = Patient(
        name=name,
        age=age,
        height=height,
        weight=weight,
        lab_results=lab_results or {},
        doctors_notes=doctors_notes,
        severity=severity
    )

    result = patient_manager.add_patient(patient)
    return result

async def async_get_patient_info(patient_id):
    patient = patient_manager.get_patient(patient_id)
    if not patient:
        return {"success": False, "error": "Patient not found"}

    return {"success": True, "patient": patient.to_dict()}

async def async_get_all_patients_info(skip=0, limit=100):
    patients = patient_manager.get_all_patients(skip, limit)
    return {
        "success": True,
        "patients": [p.to_dict() for p in patients],
        "total": patient_manager.count_patients(),
        "skip": skip,
        "limit": limit
    }

async def async_update_patient_info(patient_id, update_data):
    result = patient_manager.update_patient(patient_id, update_data)
    return result

async def async_delete_patient_record(patient_id):
    result = patient_manager.delete_patient(patient_id)
    return result

async def async_search_patients(query):
    patients = patient_manager.search_patients(query)
    return {
        "success": True,
        "patients": [p.to_dict() for p in patients],
        "count": len(patients)
    }

async def async_filter_patients(criteria):
    patients = patient_manager.filter_patients(criteria)
    return {
        "success": True,
        "patients": [p.to_dict() for p in patients],
        "count": len(patients)
    }

In [ ]:
# Define Pydantic models for API request/response validation
class LabResults(BaseModel):
    cholesterol: Optional[int] = None
    glucose: Optional[int] = None
    blood_pressure: Optional[str] = None

    class Config:
        extra = "allow"  # Allow additional fields not defined in the model

class PatientCreate(BaseModel):
    name: str = Field(..., description="Patient's full name")
    age: int = Field(..., ge=0, le=120, description="Patient's age (0-120)")
    height: float = Field(..., ge=0, le=300, description="Patient's height in cm (0-300)")
    weight: float = Field(..., ge=0, le=500, description="Patient's weight in kg (0-500)")
    lab_results: Optional[Dict] = Field(default=None, description="Laboratory test results")
    doctors_notes: Optional[str] = Field(default="", description="Doctor's notes about the patient")
    severity: str = Field(default="low", description="Patient severity (low, medium, high)")

    @validator('severity')
    def validate_severity(cls, v):
        if v not in ["low", "medium", "high"]:
            raise ValueError("Severity must be one of: low, medium, high")
        return v

class PatientUpdate(BaseModel):
    name: Optional[str] = None
    age: Optional[int] = Field(None, ge=0, le=120)
    height: Optional[float] = Field(None, ge=0, le=300)
    weight: Optional[float] = Field(None, ge=0, le=500)
    lab_results: Optional[Dict] = None
    doctors_notes: Optional[str] = None
    severity: Optional[str] = None

    @validator('severity')
    def validate_severity(cls, v):
        if v is not None and v not in ["low", "medium", "high"]:
            raise ValueError("Severity must be one of: low, medium, high")
        return v

class PatientResponse(BaseModel):
    patient_id: str
    name: str
    age: int
    height: float
    weight: float
    lab_results: Dict
    doctors_notes: str
    severity: str

class PatientListResponse(BaseModel):
    success: bool
    patients: List[PatientResponse]
    total: int
    skip: int
    limit: int

class PatientSearchResponse(BaseModel):
    success: bool
    patients: List[PatientResponse]
    count: int

class FilterCriteria(BaseModel):
    min_age: Optional[int] = None